# <font color = green> Affine gap local alignment </font>


Локальное выравнивание - это выравнивание последовательностей, не гомологичных по всей длине.

На вход подаются две последовательности (в данном решении не будет рассматриваться множественное выравнивание, во всяком случае, пока), а также целочисленные значения штрафов, которые будут начисляться за несовпадение (-s), открытие разрыва (-d_open) и продолжение разрыва (-d_ext), и значение очков за совпадения (+m).


<font size = 3> Вес выравнивания (Score) оценивается следующим образом: </font>


<font size = 2> (Количество совпадений)*m - (число несовпадений)*s - (разрывы)*d


Для локального совпадения выступающие концы не рассматриваются как пропуски (делеции)


<font size = 2> Цель данного алгоритма - найти подстроки, имеющие максимальный вес выравнивания

In [ ]:
#использованные входные данные:
#S1: tgttacgg
#S2: ggttgacta
#match = 3
#mismatch = 3
#gap_open = 2
#gap_ext = 1

S1 = input()
S2 = input()
n, m = len(S1), len(S2)

match = int(input()) #очки, прибавляемые за совпадения символов
mismatch = int(input()) # штраф за несовпадение символов
gap_open = int(input()) # штраф за открытие разрыва
gap_ext = int(input()) # штраф за продолжение разрыва

def score(i, j):
    m, x, y = 0, 0, 0
    k = abs(n - m)
    if S1[i-1] == S2[j-1]:
        m = max(M[i-1][j-1], X[i-1][j-1], Y[i-1][j-1]) + match
    else:
        m = max(M[i-1][j-1], X[i-1][j-1], Y[i-1][j-1]) - mismatch
        #if k > j:
            #k = j
        x = max((M[i][j-1] - gap_open), (Y[i][j-1] - gap_ext))
        #if k > i:
           # k = i
        y = max((M[i-1][j] - gap_open), (X[i-1][j] - gap_ext))
    max_ = max(m, x, y)
    return max_, x, y

M = [[0]*(n+1) for i in range(m+1)] #создаем матрицу из 0
X = [[0]*(n+1) for i in range(m+1)] #ending with space in X
Y = [[0]*(n+1) for i in range(m+1)] #ending with space in Y

#for i in range(1, n): #заполняем матрицы
#    for j in range(1, m):
#        M[i][j], X[i][j], Y[i][j] = score(i, j)
        
#for j in range(1, m):
#    for i in range(1, n):
#        M[i][j], X[i][j], Y[i][j] = score(i, j)
        
for i in range(1, n): #заполняем матрицу замен
    for j in range(1, m):
        M[i][j], X[i][j], Y[i][j] = score(i, j)

for l1 in M:    #конвертируем ячейки матрицы из чисел в строки для записи в файл
    for l2 in range(len(l1)):
        l1[l2] = str(l1[l2])
        
with open('agla_matrix.txt', 'w') as f:
    for row in M:
        f.write('\t'.join(row) +'\n')
        
#матрица есть, теперь нужен трейсбэк
#для этого нужно найти максимальную ячейку в матрице, и далее двигаться от неё
end = 0
end_ind = [0, 0]
for i in range(len(M)): #реализуем однопроходный алгоритм поиска максимума и его индексов
    for j in range(len(M[i])):
        M[i][j] = int(M[i][j]) #заодно и из элементов матрицы понаделаем чиселок
        if M[i][j] > end:
            end = M[i][j]
            end_ind[0], end_ind[1] = i+1, j+1
print('{} с координатами {}, {}'.format(end, end_ind[0], end_ind[1])) #нашли!

S1_aligned = ''
S2_aligned = '' #заводим строки, с помощью которых визуализируем выравнивание
score_ = end #вес выравнивания
while end != 0:
   #выбираем из соседних трех ячеек ту, в которой максимальное количество очков
    max_ = max(M[end_ind[0]-1][end_ind[1]-1], M[end_ind[0]-1][end_ind[1]], M[end_ind[0]][end_ind[1]-1])
    if max_ == M[end_ind[0]-1][end_ind[1]-1]: #ячейка по диагонали
        S1_aligned = S1_aligned + S1[end_ind[0]-1] #добавляем символы из первоначальной строк...
        S2_aligned = S2_aligned + S2[end_ind[1]-1]
        end_ind[0], end_ind[1] = end_ind[0]-1, end_ind[1]-1
    elif max_ == M[end_ind[0]-1][end_ind[1]]: #ячейка сверху
        S1_aligned = S1_aligned + '-'  #...или прочерк, если разрыв
        S2_aligned = S2_aligned + S2[end_ind[1]-1]
        end_ind[0] = end_ind[0]-1
    elif max_ == M[end_ind[0]][end_ind[1]-1]:
        S1_aligned = S1_aligned + S1[end_ind[0]-1]
        S2_aligned = S2_aligned + '-'
        end_ind[1] = end_ind[1] -1
    end = max_
    print(end)
    
S1_aligned = S1_aligned[::-1]
S2_aligned = S2_aligned[::-1] #отзеркаливаем, т.к. шли справа налево
    
with open('alignment.txt', 'w', encoding='utf-8') as f:
    f.write(' '.join(S1_aligned) +'\n')
    f.write(' '.join(S2_aligned) +'\n')
    f.write('Alignment score = {}'.format(score_) +'\n')